In [1]:
from langchain_core.prompts import PromptTemplate
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage
import base64

In [2]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

In [3]:
vision_llm = ChatOllama(model="gemma3:12b", temperature=0., num_predict=4096)

In [4]:
def image_analyzer(image_path: str) -> str:
    encoded_image = encode_image(image_path)
    message = HumanMessage(
        content=[
            {"type": "text", "text": "analyze the picture, then list ALL objects in JSON format."},
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{encoded_image}"
                },
            },
        ]
    )
    response = vision_llm.invoke([message])
    return response.content

In [5]:
print(image_analyzer("img/img.png"))

Okay, this is a challenging one! The image is incredibly dense and detailed. I'll do my best to identify as many objects as possible and present them in JSON format.  Please note that due to the complexity and overlapping nature of the image, some identifications might be approximate or open to interpretation.  I'm also prioritizing the most distinct and recognizable items.

```json
{
  "image_description": "A highly detailed, chaotic composition of various objects, primarily electronic devices, toys, and household items, arranged in a dense, layered fashion.",
  "objects": [
    {
      "category": "Vehicles",
      "items": [
        {"name": "Yellow Taxi Cab", "quantity": 1},
        {"name": "Silver/Grey Sedan", "quantity": 1},
        {"name": "Blue/White Jeep", "quantity": 1},
        {"name": "Airplane", "quantity": 1}
      ]
    },
    {
      "category": "Electronics",
      "items": [
        {"name": "Computer Monitor", "quantity": 1},
        {"name": "Computer Keyboard", 

In [6]:
prompt_template = PromptTemplate(
    input_variables=["image_description", "marketing_claim", "marketing_description"],
    template=(
        """Agisci come un consulente di marketing con 10 anni di esperienza.
Hai ricevuto la seguente descrizione di una campagna:
Claim: {marketing_claim}
Descrizione: {marketing_description}

Hai anche ricevuto la seguente descrizione dell'immagine da analizzare:
{image_description}

In base a claim e descrizione, suggerisci come migliorare l'immagine per renderla più efficace per la campagna marketing.
Fornisci consigli specifici e concreti (es. aggiungere elementi grafici, cambiare colori, inserire testo, modificare l'angolazione, inserire persone o contesto).
Non inserire preamboli e conclusioni, limitati a descrivere dei consigli specifici e concreti per migliorare l'immagine.
"""
    )
)

In [7]:
# Catena per generare i suggerimenti
suggestions_chain = prompt_template | vision_llm

In [10]:
# Ora simuliamo l'input
image_path = "img/caffe.png"  # Immagine di input
marketing_claim = "Il miglior caffè artigianale per iniziare la giornata"
marketing_description = "Una miscela premium di caffè biologico, appena tostato, ideale per chi ama l'aroma intenso e il gusto pieno. La campagna punta a trasmettere l'idea di un prodotto esclusivo, raffinato e adatto a chi vuole distinguersi."

In [11]:
# Ora generiamo i suggerimenti finali
suggestions = suggestions_chain.invoke({
    "image_description": image_analyzer(image_path),
    "marketing_claim": marketing_claim,
    "marketing_description": marketing_description
})

print("Suggerimenti per migliorare l'immagine:")
print(suggestions.content)

Suggerimenti per migliorare l'immagine:
*   **Eliminare o attenuare il roaster:** La presenza del roaster, anche parziale, suggerisce un processo industriale. Il claim punta all'artigianalità. Rimuoverlo completamente o sfocarlo ulteriormente per renderlo meno evidente.
*   **Sostituire la scoop con una in legno:** Il colore blu della scoop metallica non si allinea con l'idea di artigianalità e raffinatezza. Una scoop in legno, magari con finiture curate, si integrerebbe meglio con l'estetica desiderata.
*   **Modificare il colore dell'abbigliamento:** Il rosso/burgundy dell'abbigliamento è troppo vivace e distrae. Sostituirlo con tonalità più neutre e calde come beige, crema o marrone chiaro, per creare un'atmosfera più rilassata e sofisticata.
*   **Migliorare l'illuminazione:** L'illuminazione dovrebbe essere calda e soffusa, per esaltare i colori dei chicchi di caffè e creare un'atmosfera accogliente. Evitare ombre nette e luci dirette.
*   **Aggiungere elementi di contesto:** Inse